In [7]:
import quantiacsToolbox
import pandas as pd
import numpy as np
import os
from os.path import dirname, abspath
import matplotlib.pyplot as plt
from pmdarima.arima import auto_arima
import pickle
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.layers import Dropout
import tensorflow
from keras.models import model_from_json
from keras.preprocessing.sequence import TimeseriesGenerator

In [8]:
markets = ['CASH','F_AD','F_BO','F_BP','F_C','F_CC','F_CD','F_CL','F_CT','F_DX','F_EC',
                           'F_ED','F_ES','F_FC','F_FV','F_GC','F_HG','F_HO','F_JY','F_KC','F_LB','F_LC',
                           'F_LN','F_MD','F_MP','F_NG','F_NQ','F_NR','F_O','F_OJ','F_PA','F_PL','F_RB',
                           'F_RU','F_S','F_SB','F_SF','F_SI','F_SM','F_TU','F_TY','F_US','F_W','F_XX',
                           'F_YM','F_AX','F_CA','F_DT','F_UB','F_UZ','F_GS','F_LX','F_SS','F_DL','F_ZQ',
                           'F_VX','F_AE','F_BG','F_BC','F_LU','F_DM','F_AH','F_CF','F_DZ','F_FB','F_FL',
                           'F_FM','F_FP','F_FY','F_GX','F_HP','F_LR','F_LQ','F_ND','F_NY','F_PQ','F_RR',
                           'F_RF','F_RP','F_RY','F_SH','F_SX','F_TR','F_EB','F_VF','F_VT','F_VW','F_GD','F_F']

In [9]:
train_begin = '20180101'
train_end = '20201231'
df = pd.read_csv('data_'+train_begin+'_'+train_end+'.csv')
df = df.rename(columns = {'Unnamed: 0':'date'})
df['date'] = pd.to_datetime(df['date'])
df = df.set_index('date')

In [10]:
correct_dict = {el:None for el in markets[1:]}

In [11]:
def prepare_columns(data):
    df = pd.DataFrame(data)
    columns, names = list(), list()
    # input sequence
    for i in range(5, 0, -1):
        columns.append(df.shift(i))
        names += [('var(t-%d)' % (i))]
    # forecast sequence
    columns.append(df)
    names += [('var(t)')]
    con = pd.concat(columns, axis=1)
    con.columns = names
    con.dropna(inplace=True)
    return con

In [12]:
def prepare_data(scaler, data):
    values = data.values
    values = values.reshape(len(values), 1)
    scaled = scaler.fit_transform(values)
    scaled = scaled.reshape(len(scaled), 1)
    final = prepare_columns(scaled)
    final = final.values
    return final

In [13]:
def forecast_lstm(model, test):
    forecasts = list()
    for i in range(len(test)):
        X, Y = test[i, 0:5], test[i, 5:]
        X = X.reshape(1, 1, len(X))
        pred = model.predict(X, batch_size=1)
        fore = [x for x in pred[0, :]]
        forecasts.append(fore)
    return forecasts

In [14]:
def inverse(series, forecasts, scaler):
    inverted = list()
    for i in range(len(forecasts)):
        forecast = np.array(forecasts[i])
        forecast = forecast.reshape(1, len(forecast))
        inv_scale = scaler.inverse_transform(forecast)
        inv_scale = inv_scale[0, :]
        inverted.append(inv_scale)
    return inverted

In [23]:
for market in markets[3:4]:
    correct = 0
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaler.fit_transform(df[[market]])
    print(market)
    for i in range(len(df)-100, len(df)-1):
        test = prepare_data(scaler, df[[market]].loc[:df.index[i],:])
        json_file = open('multi_step_lstm_models/multi_step_lstm_model_'+market+'.json', 'r')
        loaded_model_json = json_file.read()
        json_file.close()
        model = model_from_json(loaded_model_json)
        model.load_weights('multi_step_lstm_models/multi_step_lstm_model_'+market+'.h5')
        forecasts = forecast_lstm(model, test)
        forecasts = inverse(df[[market]].loc[:df.index[i],:], forecasts, scaler)
        actual = [row[5:] for row in test]
        actual = inverse(df[[market]], actual, scaler)
        pred = [fore[0] for fore in forecasts]
        pos = 1 if pred[-1] > df[market][i] else -1
        act = 1 if df[market][i+1] > df[market][i] else -1
        if pos == act:
            correct += 1
    correct_dict[market] = correct